# Mining Data Streams


## Sampling

- Reservoir
    fixed size
    pick new sample with probability s/n
    replace uniformly, all s samples are equally likely to get replaced

## Filtering
- First cut
    hashed bit array
- Bloom filter
    using n hash functions to verify that x is in S
    optimal k = n/m *ln(2)

## Algorithms on stream
- unique elements
- occurence
    - moments
        - 0 - distinct values
        - 1 - stream length
        - 2 - surprise
        - k - Sum of (m_j)^k = n(c_k - ( c - 1)^k )




implementing: [In-Core Computation of Geometric Centralities with
HyperBall:
A Hundred Billion Nodes and Beyond](https://arxiv.org/pdf/1308.2144v2)
1. Flajolet-Martin algorithm
2. Streaming graph algorithm



HyperLogLog

HyperBall


In [1]:
# HyperLogLog counter

from mmh3 import hash

def _hash(x):
    return hash(x)

def _leading_zeros(x):
    if isinstance(x, bytes):
        # Convert bytes to string and count leading zeros
        return str(x, 'ascii').count('0', 0, str(x, 'ascii').find('1'))
    elif isinstance(x, int):
        # Convert integer to binary string and count leading zeros
        return bin(x)[2:].count('0', 0, bin(x)[2:].find('1'))

    raise ValueError(f"Unsupported type: {type(x)}")

def p_plus_one(x):
    return _leading_zeros(x) + 1

class Counter:
    def __init__(self, b):
        self.p = (2**b)
        self.b = b
        self.M = [0] * self.p
        self.alpha_p = 0.7213 / (1 + 1.079 / self.p) # https://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf p.140 (14)
    
    def set_m(self, m):
        self.M = m

    def add(self, x, verbose=False):
        h = _hash(str(x).encode('utf-8'))
        # Get bucket index using first b bits
        i = h & ((1 << self.b) - 1)
        # Get remaining bits
        w = h >> self.b
        # count leading zeros
        p = p_plus_one(w)
        self.M[i] = max(self.M[i], p)
        
        if verbose: print(f"h: {h:032b}, i: {i:{self.b}b}, w: {w:{32-self.b}b}, p_+1: {p}, M[i]: {self.M[i]}")


    def size(self):
        Z = sum(2**(-m) for m in self.M)
        return self.alpha_p * self.p**2 * Z
    
    def _repr_(self):
        return f"Counter(b={self.b}, p={self.p}, M={self.M})"
    
    


In [2]:
c = Counter(4)

In [3]:
c.add(500, True)
c.size()


h: 00000110000101001011110011100110, i:  110, w:      11000010100101111001110, p_+1: 1, M[i]: 1


2681.2983429943206

In [4]:
c._repr_()

'Counter(b=4, p=16, M=[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])'

In [5]:
c.add(100)
c.size()


2594.8048480590196

In [6]:
c._repr_()

'Counter(b=4, p=16, M=[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1])'

In [7]:
def process_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            if line.strip().startswith('#'):
                continue
            source, target = map(int, line.strip().split())
            data.append((source, target))
    return data
    
class Graph:
    def __init__(self, data):
        self.build(data)
    
    def build(self, data):
        self.adj_list = {}
        self.nodes = set()
        
        # Build the graph
        for source, target in data:
            self.nodes.add(source)
            self.nodes.add(target)
            
            if source not in self.adj_list:
                self.adj_list[source] = set()
            self.adj_list[source].add(target)
            
        # Add node mapping
        self.node_to_idx = {node: idx for idx, node in enumerate(sorted(self.nodes))}
        self.idx_to_node = {idx: node for node, idx in self.node_to_idx.items()}
    
    def neighbors(self, idx):
        """Return neighbors of a node (using internal index)"""
        node = self.idx_to_node[idx]
        return {self.node_to_idx[n] for n in self.adj_list.get(node, set())}
    
    def num_nodes(self):
        """Return total number of nodes"""
        return len(self.nodes)
        
        


In [8]:
data = process_data('data/web-BerkStan.txt')

In [9]:

G = Graph(data[:1000])


In [12]:


import os

class Disk:
    def __init__(self, path):
        self.path = path
        self.delimiter = '---'
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(self.path), exist_ok=True)
        
        # create path if doesn't exist
        if not os.path.exists(self.path):
            with open(self.path, 'w') as f:
                f.write('')
        
        self.clear()
    
    def clear(self):
        with open(self.path, 'w') as f:
            f.write('')

    def write_single_vc(self, v, c):
        """Write a single edge to disk"""
        with open(self.path, 'a') as f:
            # Convert list to comma-separated string
            c_str = ','.join(map(str, c))
            f.write(f"{v}{self.delimiter}{c_str}\n")
    
    def read_edges(self):
        """Read edges from disk and return as list of tuples"""
        edges = []
        with open(self.path, 'r') as f:
            for line in f:
                v, c_str = line.strip().split(self.delimiter)
                # Convert string back to list of integers
                c = list(map(int, c_str.split(',')))
                edges.append((int(v), c))
        return edges

def union(a:Counter, b:Counter):
    for i in range(a.p):
        a.M[i] = max(a.M[i], b.M[i])
    return a


# HyperBall algorithm
D = Disk('tmp/counts.txt')
B = 10
N = G.num_nodes()

# intialize counters
c = [Counter(B) for _ in range(N)]

# add self to each counter
for v in range(N):
    c[v].add(v)

while True:
    for v in range(N):
        a = c[v]
        for w in G.neighbors(v):
            a = union(c[w], a)
        
        # write to disk
        D.write_single_vc(v, a.M)
    
    # read from disk
    vms = D.read_edges()
    D.clear()
    
    # update counters if changed
    changed = False
    for v, m in vms:
        if c[v].M != m:
            c[v].set_m(m)
            changed = True
    
    if not changed:
        break
    


KeyboardInterrupt: 